<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/yfinance_bist_bilanco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Gerekli kütüphaneler
!pip install investpy xlsxwriter

import investpy
import yfinance as yf
import pandas as pd
import xlsxwriter
from requests.exceptions import HTTPError

# Tüm BIST hisselerini çekelim
turkish_stocks = investpy.stocks.get_stocks(country='Turkey')
hisseler = turkish_stocks['symbol'].tolist()
hisse_listesi = [hisse + ".IS" for hisse in hisseler]

veriler = []
hatali_hisseler = []

print("Tarama başladı... devam ediyor")
for hisse in hisse_listesi:
    try:
        ticker = yf.Ticker(hisse)
        finansal_veri = ticker.info
        balance_sheet = ticker.balance_sheet
        income_statement = ticker.financials
        cashflow = ticker.cashflow

        if not finansal_veri or "quoteType" not in finansal_veri:
            continue
        if balance_sheet.empty:
            continue
        if income_statement.empty:
            continue
        if cashflow.empty:
            continue

        # Finansal metrikler
        firma_degeri = finansal_veri.get("enterpriseValue", 0)
        piyasa_degeri = finansal_veri.get("marketCap", 0)
        nakit = finansal_veri.get("totalCash", 0)
        defter_degeri = finansal_veri.get("bookValue", 0)
        ebitda = income_statement.loc['EBITDA'].iloc[0] if 'EBITDA' in income_statement.index else None
        net_kar = finansal_veri.get("netIncomeToCommon", 0)
        toplam_gelir = income_statement.loc['Total Revenue'].iloc[0] if 'Total Revenue' in income_statement.index else None
        toplam_borc = balance_sheet.loc['Total Debt'].iloc[0] if 'Total Debt' in balance_sheet.index else None
        hisse_basi_gelir = finansal_veri.get("revenuePerShare", 0)
        son_ceyrek_kar = finansal_veri.get("earningsQuarterlyGrowth", 0)
        brut_kar_marji = finansal_veri.get("grossMargins", 0)
        brut_kar = income_statement.loc['Gross Profit'].iloc[0] if 'Gross Profit' in income_statement.index else None
        faaliyet_kar_marji = finansal_veri.get("operatingMargins", 0)
        varlik_karliligi = finansal_veri.get("returnOnAssets", 0)
        ozsermaye_karliligi = finansal_veri.get("returnOnEquity", 0)
        fk_trailing = finansal_veri.get("trailingPE", 0)
        fk_forward = finansal_veri.get("forwardPE", 0)
        hizli_oran = finansal_veri.get("quickRatio", 0)
        borc_ozkaynak_orani = finansal_veri.get("debtToEquity", 0)
        cari_oran = finansal_veri.get("currentRatio", 0)
        #net_borc = toplam_borc - nakit
        pd_dd = ticker.info.get('marketCap') / ticker.info.get('bookValue')
        peg_orani = finansal_veri.get("pegRatio", 0)
        operating_cashflow = finansal_veri.get("operatingCashflow", None)
        sermaye_harcamalari = finansal_veri.get("capitalExpenditures", None)
        serbestnakitakisi = finansal_veri.get("freeCashflow", None)
        ozkaynak = finansal_veri.get("totalStockholderEquity", None)
        uzunvadeliborc = finansal_veri.get("longTermDebt", None)
        kisavadeborc = finansal_veri.get("totalCurrentLiabilities", None)
        fd_favok = ticker.info['enterpriseValue'] / ticker.info['ebitda'] if ticker.info['ebitda'] else None
        #fd_favok = ticker.info.get('enterpriseValue', None) / ticker.info.get('ebitda', None) if ticker.info.get('ebitda', None) else None
        toplam_varliklar = balance_sheet.loc['Total Assets'].iloc[0] if 'Total Assets' in balance_sheet.index else None
        donen_varliklar = balance_sheet.loc['Current Assets'].iloc[0] if 'Current Assets' in balance_sheet.index else None
        toplam_yukumlulukler = balance_sheet.loc['Total Liabilities Net Minority Interest'].iloc[0] if 'Total Liabilities Net Minority Interest' in balance_sheet.index else None
        donen_yukumlulukler = balance_sheet.loc['Current Liabilities'].iloc[0] if 'Current Liabilities' in balance_sheet.index else None
        duran_yukumlulukler = balance_sheet.loc['Total Non Current Liabilities Net Minority Interest'].iloc[0] if 'Total Non Current Liabilities Net Minority Interest' in balance_sheet.index else None
        azinlik_hakki = balance_sheet.loc['Minority Interest'].iloc[0] if 'Minority Interest' in balance_sheet.index else None
        net_borc = balance_sheet.loc['Net Debt'].iloc[0] if 'Net Debt' in balance_sheet.index else None
        faaliyet_gideri = income_statement.loc['Operating Expense'].iloc[0] if 'Operating Expense' in income_statement.index else None
        faaliyet_geliri = income_statement.loc['Operating Income'].iloc[0] if 'Operating Income' in income_statement.index else None
        net_gelir = income_statement.loc['Net Income'].iloc[0] if 'Net Income' in income_statement.index else None
        faiz_gideri = income_statement.loc['Interest Expense'].iloc[0] if 'Interest Expense' in income_statement.index else None
        vergi_oncesi_kar = income_statement.loc['Pretax Income'].iloc[0] if 'Pretax Income' in income_statement.index else None
        hisse_basina_kar = income_statement.loc['Diluted EPS'].iloc[0] if 'Diluted EPS' in income_statement.index else None


        # Net Kar Marjı (%) hesapla
        if toplam_gelir != 0:
            net_kar_marji = (net_kar / toplam_gelir) * 100
        else:
            net_kar_marji = 0

        veriler.append({
            "Hisse": hisse,
            "Sektör": finansal_veri.get("sector", "N/A"),
            "Analist hedef fiyat": finansal_veri.get("targetMeanPrice", 0),
            "Firma Değeri (₺)": firma_degeri,
            "Dolaşımdaki Hisse Adedi": finansal_veri.get("sharesOutstanding", 0),
            "Piyasa Değeri (₺)": piyasa_degeri,
            "Defter Değeri (₺)": defter_degeri,
            "PD/DD": pd_dd,
            "FAVÖK (₺)": ebitda,
            "FD/FAVÖK": fd_favok,
            "EBITDA (₺)": ebitda,
            "Çeyrek Kâr Büyümesi (%)": son_ceyrek_kar,
            "Kâr Büyümesi (%)": finansal_veri.get("earningsGrowth", 0),
            "Gelir Büyümesi (%)": finansal_veri.get("revenueGrowth", 0),
            "Varlık Karlılığı (ROA) (%)": varlik_karliligi,
            "Özsermaye Karlılığı (ROE) (%)": ozsermaye_karliligi,
            "Net Kar Marjı (%)": net_kar_marji,
            "Borç/Özkaynak Oranı": borc_ozkaynak_orani,
            "FK (Trailing PE)": fk_trailing,
            "FK (Forward PE)": fk_forward,
            "Nakit (₺)": nakit,
            "Net Borç (₺)": net_borc,
            "Toplam Borç (₺)": toplam_borc,
            "Hisse Başına Gelir (₺)": hisse_basi_gelir,
            "Brüt Kar Marjı": brut_kar_marji,
            "Toplam Gelir": toplam_gelir,
            "Faaliyet Geliri (₺)": faaliyet_geliri,
            "Faaliyet Kar Marjı (%)": faaliyet_kar_marji,
            "Faaliyet Gideri (₺)": faaliyet_gideri,
            "Toplam Varlıklar (₺)": toplam_varliklar,
            "Dönen Varlıklar (₺)": donen_varliklar,
            "Toplam Yükümlülükler (₺)": toplam_yukumlulukler,
            "Dönen Yükümlülükler (₺)": donen_yukumlulukler,
            "Duran Yükümlülükler (₺)": duran_yukumlulukler,
            "Kısa Vadeli Yükümlülükler (₺)": kisavadeborc,
            "Uzun Vadeli Borçlar (₺)": uzunvadeliborc,
            "Özkaynak (₺)": ozkaynak,
            "Azınlık Hakkı (₺)": azinlik_hakki,
            "Net Borç (₺)": net_borc,
            "Net Gelir (₺)": net_gelir,
            "Faiz Gideri (₺)": faiz_gideri,
            "Vergi Öncesi Kâr (₺)": vergi_oncesi_kar,
            "Brüt Kâr (₺)": brut_kar,
            "Net Kâr (₺)": net_kar,
            "Seyreltilmiş Hisse Başına Kâr (₺)": hisse_basina_kar,
            "İşletme Faaliyetlerinden Gelen Nakit Akışı (₺)": operating_cashflow,
            "Sermaye Harcamaları (₺)": sermaye_harcamalari,
            "Serbest Nakit Akışı (₺)": serbestnakitakisi,
            "Cari Oran": cari_oran,
            "Hızlı Oran": hizli_oran,
            "Temettü Getirisi (%)": finansal_veri.get("dividendYield", 0),
            "Temettü Oranı": finansal_veri.get("dividendRate", 0),
        })
    except HTTPError as http_err:
        if "404" not in str(http_err):
            hatali_hisseler.append(hisse)
    except Exception as e:
        hatali_hisseler.append(hisse)

# DataFrame
print("Tarama tamamlandı. Veriler sıralanıyor...")
df = pd.DataFrame(veriler)
df = df.sort_values(by="Net Kâr (₺)", ascending=False)


# Excel
from datetime import datetime
tarih = datetime.now().strftime("%Y-%m-%d")
dosya_adi = f"Finansal_Veriler_{tarih}.xlsx"

with pd.ExcelWriter(dosya_adi, engine='xlsxwriter') as writer:
    df['Hisse'] = df['Hisse'].str.replace('.IS', '', regex=False)
    df.to_excel(writer, index=False, sheet_name='Finansal Veriler')
    workbook = writer.book
    worksheet = writer.sheets['Finansal Veriler']
    worksheet.set_zoom(85)

    number_format = workbook.add_format({'num_format': '#,##0.00'})
    for idx, col in enumerate(df.columns):
        if df[col].dtype in ['float64', 'int64']:
            worksheet.set_column(idx, idx, 18, number_format)

# Anlık Görüntüleme
print("Finansal veriler başarıyla kaydedildi ve sıralandı.")
display(df)

print(f"Tarama bitti. Başarısız hisseler: {len(hatali_hisseler)}")